# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,12.95,88,0,3.60,ZA,1722543588
1,1,ushuaia,-54.8000,-68.3000,3.81,87,20,4.63,AR,1722543589
2,2,polyarnyy,69.1989,33.4478,9.96,95,99,4.78,RU,1722543590
3,3,puerto ayora,-0.7393,-90.3518,26.98,85,70,4.80,EC,1722543591
4,4,la passe,45.5549,-0.8967,23.13,81,0,3.95,FR,1722543592
5,5,greymouth,-42.4667,171.2000,-0.05,89,7,3.85,NZ,1722543593
6,6,kodiak,57.7900,-152.4072,14.73,63,0,1.54,US,1722543594
7,7,namsos,64.4662,11.4957,13.95,77,75,1.54,NO,1722543596
8,8,larkird,35.4894,66.6641,16.66,37,0,1.74,AF,1722543597
9,9,turbat,26.0023,63.0440,29.08,77,100,4.10,PK,1722543598


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    coastline = True,
    hover_cols='all',
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity,index,City_ID,Max Temp,Cloudiness,Wind Speed,Country,Date)
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,la passe,45.5549,-0.8967,23.13,81,0,3.95,FR,1722543592
196,196,labarthe-sur-leze,43.4521,1.3997,24.90,69,0,3.60,FR,1722543802
235,235,arica,-18.4750,-70.3042,21.75,67,0,3.60,CL,1722543844
256,256,barcelos,41.5388,-8.6151,21.06,90,0,3.15,PT,1722543866
273,273,ribeira brava,32.6500,-17.0667,25.40,83,0,3.06,PT,1722543884
276,276,mahina,-17.5065,-149.4890,25.87,50,0,2.06,PF,1722543887
292,292,borgo san dalmazzo,44.3330,7.4858,24.41,81,0,1.34,IT,1722543904
304,304,damghan,36.1683,54.3480,26.34,46,0,3.49,IR,1722543916
422,422,moires,35.0532,24.8744,25.42,52,0,1.79,GR,1722544046
465,465,liaocheng,36.4439,115.9647,26.19,95,0,3.87,CN,1722544092


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

C:\Users\ambmd\AppData\Local\Temp\ipykernel_3468\3112125336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,la passe,FR,45.5549,-0.8967,81,
196,labarthe-sur-leze,FR,43.4521,1.3997,69,
235,arica,CL,-18.4750,-70.3042,67,
256,barcelos,PT,41.5388,-8.6151,90,
273,ribeira brava,PT,32.6500,-17.0667,83,
276,mahina,PF,-17.5065,-149.4890,50,
292,borgo san dalmazzo,IT,44.3330,7.4858,81,
304,damghan,IR,36.1683,54.3480,46,
422,moires,GR,35.0532,24.8744,52,
465,liaocheng,CN,36.4439,115.9647,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
from pprint import pprint

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']

    # Add the current city's latitude and longitude to the params dictionary
    params["text"] = city
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la passe - nearest hotel: L'Estuaire
labarthe-sur-leze - nearest hotel: Hôtel Clément Ader
arica - nearest hotel: Hotel Andalucía 
barcelos - nearest hotel: Hotel Dom Nuno
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
mahina - nearest hotel: Motu Martin
borgo san dalmazzo - nearest hotel: Hotel Ristorante Roma
damghan - nearest hotel: هتل جهانگردى دامغان
moires - nearest hotel: Hotel Olympic
liaocheng - nearest hotel: 聊城荣逸酒店
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
sant'antioco - nearest hotel: Hotel Café del Corso
obiliq - nearest hotel: Hotel Castel
beira - nearest hotel: Hotel Savoy


,City,Country,Lat,Lng,Humidity,Hotel Name
4,la passe,FR,45.5549,-0.8967,81,L'Estuaire
196,labarthe-sur-leze,FR,43.4521,1.3997,69,Hôtel Clément Ader
235,arica,CL,-18.4750,-70.3042,67,Hotel Andalucía
256,barcelos,PT,41.5388,-8.6151,90,Hotel Dom Nuno
273,ribeira brava,PT,32.6500,-17.0667,83,Cheerfulway Bravamar Hotel
276,mahina,PF,-17.5065,-149.4890,50,Motu Martin
292,borgo san dalmazzo,IT,44.3330,7.4858,81,Hotel Ristorante Roma
304,damghan,IR,36.1683,54.3480,46,هتل جهانگردى دامغان
422,moires,GR,35.0532,24.8744,52,Hotel Olympic
465,liaocheng,CN,36.4439,115.9647,95,聊城荣逸酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    coastline = True,
    hover_cols='all',
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)
   .Coastline.I :Feature   [Longitude,Latitude]